In [2]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio



# mapping to the modules that make the app
sys.path.insert(0, "../..")



In [14]:
from app.objects import structures
from notebooks.helpers import test_queries

In [15]:
ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import app.connectors.cmdb_graph as cmdb_graph

c = cmdb_graph.CosmosdbClient()

In [16]:
buildings = yaml.safe_load(
    open(os.path.join(os.getenv("abspath"), "app", "configurations", "buildings.yaml"))
)
buildings["explanation"]

[{'type': 'id_name of the building. lower, underscores'},
 {'label': "'class' of the object. in this case it is always 'building'"},
 {'name': "pretty printed name of the building. UI can remove the '_' at display time."},
 {'augments_resource': 'OPTIONAL augments a specific resource, creates it if it does not exist'},
 {'owned_by': 'type of object that can own/maintain this building, assuming a relationhsip to that object'},
 {'faction_augments': 'if owned by a pop in the faction,  raises the stat for all pops in factions'},
 {'effort': 'time in universal time units (UTU) that the action will take to complete'},
 {'requires_attr': 'required attributes of the agent. Determines if the agent can do the action.'},
 {'requires_faction': 'requires that the faction have this attribute'},
 {'renews_faction_resource': 'augments (or diminishes) a resource in the faction. Creates if it does not exist.'},
 {'renews_location_resource': 'augments (or diminishes) a resource in the location. Creates 

# Creating a new Building

In [17]:
userguid = "8d5b667f-b225-4641-b499-73b77558ff86"
pop = test_queries.get_random_pop(c,userguid)
pop


{'name': 'Yetam Grolesmin',
 'objid': '3195718763007',
 'conformity': 0.558,
 'literacy': 0.576,
 'aggression': 0.487,
 'constitution': 0.598,
 'health': 0.7,
 'isIn': '0493222540853',
 'industry': 0.5425,
 'wealth': 0.5592,
 'factionLoyalty': 0.533,
 'isIdle': 'true',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'pop',
 'id': '3195718763007'}

In [7]:
buildings_list = list(buildings["buildings"].keys())
buildings_list

['farm',
 'hospital',
 'solar_panel',
 'commercial',
 'forestry',
 'infrastructure',
 'university',
 'observatory',
 'prospector',
 'oil_well',
 'marketplace',
 'shipyard',
 'industrial_complex',
 'flight_control',
 'launch_complex']

Should be able to search the building type by name

In [7]:
b = buildings["buildings"]['farm']
b

{'type': 'farmland',
 'label': 'building',
 'name': 'Farmland',
 'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
 'planet_requirements': {'isHabitable': 'true'},
 'faction_augments': {'wealth': -1},
 'renews_faction_resource': {'grains': 10},
 'owned_by': 'pop',
 'effort': 1}

In [8]:
building = structures.Building(pop,b)
building

<building: None; 5513462776036; Farmland>

In [9]:
building.get_data()

{'name': 'Farmland',
 'objid': '5513462776036',
 'label': 'building',
 'ownedBy': '2177835361193',
 'type': 'farmland',
 'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
 'planet_requirements': {'isHabitable': 'true'},
 'faction_augments': {'wealth': -1},
 'renews_faction_resource': {'grains': 10},
 'owned_by': 'pop',
 'effort': 1}

There should be a building action. If there isn't, go to the UI and create one. 

In [10]:
c.run_query("g.V().has('label','action').valueMap()")
action = c.clean_nodes(c.res)[0]
action

{'type': 'construction',
 'comment': 'constructing a Farmland',
 'effort': 1,
 'applies_to': 'pop',
 'owned_by': 'pop',
 'building': 'farmland',
 'created_at': 65414,
 'to_build': '{type: farmland, label: building, name: Farmland, description: Generates organic foodstuffs, populations will consume food before consuming natural resources, planet_requirements: {isHabitable: true}, faction_augments: {wealth: -1}, renews_faction_resource: {grains: 10}, owned_by: pop, effort: 1}',
 'objid': '9274928873387',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'action',
 'id': '9274928873387'}

In [11]:
to_build = action['to_build']
to_build

'{type: farmland, label: building, name: Farmland, description: Generates organic foodstuffs, populations will consume food before consuming natural resources, planet_requirements: {isHabitable: true}, faction_augments: {wealth: -1}, renews_faction_resource: {grains: 10}, owned_by: pop, effort: 1}'

In [12]:
building.get_owned_by()

{'node1': '2177835361193', 'node2': '5513462776036', 'label': 'owns'}

You can arbitrarily place a building here for testing purposes. 

In [13]:
data = {'nodes':[building.get_data()],'edges':[building.get_owned_by()]}
# c.upload_data(userguid,data)
data

{'nodes': [{'name': 'Farmland',
   'objid': '5513462776036',
   'label': 'building',
   'ownedBy': '2177835361193',
   'type': 'farmland',
   'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
   'planet_requirements': {'isHabitable': 'true'},
   'faction_augments': {'wealth': -1},
   'renews_faction_resource': {'grains': 10},
   'owned_by': 'pop',
   'effort': 1}],
 'edges': [{'node1': '2177835361193',
   'node2': '5513462776036',
   'label': 'owns'}]}

In [14]:
building_query = f"""
g.V().has('objid','{pop['objid']}').as('pop')
    .out('owns').as('building')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap())
"""

c.run_query(building_query)
c.res

[]

## Constructing buildings through Action

In the UI the construction tab will create an actioni that needs to be resolved. That action is eventually caught by the `actionResolverTimer` and then passed to the `resolveActionEvents` functions. 

In [15]:
from app.objects import time
from app.functions import jobs

In [16]:
t = time.Time(c)
t.get_current_UTU()
t

< time at UTU:65559 >

We careate an action object to make it easy to manage. Just like other actions, the `weight` indicates if the time has passed. 

This is all taken care of from the `actionResolverTimer` function under 'job_messages' section.

In [22]:
job_messages = jobs.resolve_jobs(c,t,time.Action)
pd.DataFrame(job_messages)

,agent,action,job
0,"{'isIdle': 'false', 'name': 'Tadta Bil', 'obji...","{'type': 'construction', 'comment': 'construct...","{'status': 'pending', 'userguid': '8d5b667f-b2..."


The jobs above are sent to `resolveActionEvents` to resolve them asynchronosly. 

In [23]:
message = job_messages[0]
message.keys()

dict_keys(['agent', 'action', 'job'])

In [24]:
'job' in message.keys(), message['job']['actionType'] == "construction"

(True, True)

In [26]:
# structures.construct_building(c,message)

## Buildings that have buttons and can take actions

In [20]:
userguid = "8d5b667f-b225-4641-b499-73b77558ff86"
pop = test_queries.get_random_pop(c,userguid)
pop


{'name': 'Yetam Fen',
 'objid': '5657120867640',
 'conformity': 0.309,
 'literacy': 0.636,
 'aggression': 0.341,
 'constitution': 0.456,
 'health': 0.7,
 'isIn': '0493222540853',
 'industry': 0.3985,
 'wealth': 0.5172,
 'factionLoyalty': 0.539,
 'isIdle': 'true',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'pop',
 'id': '5657120867640'}

In [21]:
b = buildings["buildings"]['industrial_complex']
b

{'type': 'manufacturing',
 'label': 'building',
 'name': 'Industrial manufacturing complex',
 'description': 'Builds ships and other large objects for the faction.',
 'owned_by': 'pop',
 'effort': 10,
 'faction_augments': {'wealth': 1},
 'requires_attr': {'industry': 0.1},
 'has_buttons': ['build_ship']}

In [22]:
building = structures.Building(pop,b)
building.get_data()

{'name': 'Industrial manufacturing complex',
 'objid': '7660114240581',
 'label': 'building',
 'ownedBy': '5657120867640',
 'type': 'manufacturing',
 'description': 'Builds ships and other large objects for the faction.',
 'owned_by': 'pop',
 'effort': 10,
 'faction_augments': {'wealth': 1},
 'requires_attr': {'industry': 0.1},
 'has_buttons': ['build_ship']}

In [24]:
c.run_query("g.V().has('label','action').valueMap()")
action = c.clean_nodes(c.res)[0]
action

{'type': 'construction',
 'comment': 'constructing a Industrial manufacturing complex',
 'effort': 10,
 'applies_to': 'pop',
 'owned_by': 'pop',
 'building': 'manufacturing',
 'created_at': 67597,
 'to_build': '{type: manufacturing, label: building, name: Industrial manufacturing complex, description: Builds ships and other large objects for the faction., owned_by: pop, effort: 10, faction_augments: {wealth: 1}, requires_attr: {industry: 0.1}, has_buttons: [build_ship]}',
 'objid': '7792223898957',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'action',
 'id': '7792223898957'}

In [26]:
data = {'nodes':[building.get_data()],'edges':[building.get_owned_by()]}
c.upload_data(userguid,data)
data

{'nodes': [{'name': 'Industrial manufacturing complex',
   'objid': '7660114240581',
   'label': 'building',
   'ownedBy': '5657120867640',
   'type': 'manufacturing',
   'description': 'Builds ships and other large objects for the faction.',
   'owned_by': 'pop',
   'effort': 10,
   'faction_augments': {'wealth': 1},
   'requires_attr': {'industry': 0.1},
   'has_buttons': ['build_ship'],
   'id': '7660114240581'}],
 'edges': [{'node1': '5657120867640',
   'node2': '7660114240581',
   'label': 'owns'}]}